**Install Required Packages**

In [ ]:
!pip install pandas numpy matplotlib
!pip install ib_insync
!pip install yfinance  # Backup data source

**Install IBKR API**

In [ ]:
!pip install --upgrade pip
!pip install ibapi

# Alternative installation if needed
try:
    !git clone https://github.com/InteractiveBrokers/tws-api.git
    !cd tws-api/source/pythonclient && python setup.py install
    import sys
    sys.path.append('/content/tws-api/source/pythonclient')
except:
    print("Git clone failed, but that's okay - using pip installation")

# Verify installation
try:
    from ibapi.client import EClient
    from ibapi.wrapper import EWrapper
    from ibapi.contract import Contract
    print("✅ IBKR API components imported successfully!")
    print("✅ Installation is working correctly")
except ImportError:
    print("❌ IBKR API installation failed")

Cloning into 'tws-api'...
fatal: could not read Username for 'https://github.com': No such device or address
/bin/bash: line 1: cd: tws-api/source/pythonclient: No such file or directory
✅ IBKR API components imported successfully!
✅ Installation is working correctly


**Import Required Libraries**

In [ ]:
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import logging
import warnings
from datetime import datetime, timedelta

# Configure settings
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
plt.style.use('seaborn-v0_8')  # Better plot styling

**Adapted SMA Strategy Code for Colab**

In [ ]:
class EnhancedIBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []  # Historical data storage
        self.positions = {}  # Current positions
        self.data_received = False
        self.connection_status = False
        self.account_info = {}
        self.next_order_id = None
        self.order_status = {}

    def nextValidId(self, orderId):
        """Called when connection is established"""
        super().nextValidId(orderId)
        self.next_order_id = orderId
        self.connection_status = True
        logging.info(f"✅ Connected to TWS. Next valid order ID: {orderId}")

    def historicalData(self, reqId, bar):
        """Called when historical data is received"""
        self.data.append([
            bar.date, bar.open, bar.high,
            bar.low, bar.close, bar.volume
        ])

    def historicalDataEnd(self, reqId, start, end):
        """Called when all historical data has been received"""
        logging.info(f"📊 Historical data received: {len(self.data)} bars from {start} to {end}")
        self.data_received = True

    def position(self, account, contract, position, avgCost):
        """Called when position information is received"""
        super().position(account, contract, position, avgCost)
        key = contract.symbol
        self.positions[key] = {
            'position': position,
            'avgCost': avgCost,
            'unrealizedPNL': 0
        }
        logging.info(f"📈 Position for {key}: {position} shares at avg cost ${avgCost:.2f}")

    def accountSummary(self, reqId, account, tag, value, currency):
        """Store account information"""
        self.account_info[tag] = value

    def error(self, reqId, errorCode, errorString):
        """Enhanced error handling"""
        # Filter out common non-critical errors
        non_critical_errors = [2104, 2106, 2158, 2168, 2137]
        if errorCode not in non_critical_errors:
            if errorCode == 502:
                logging.error("❌ Connection failed - check TWS/Gateway is running")
            elif errorCode == 200:
                logging.warning(f"⚠️  Market data warning: {errorString}")
            else:
                logging.error(f"❌ Error {errorCode}: {errorString}")

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice,
                   permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        """Track order status"""
        self.order_status[orderId] = {
            'status': status,
            'filled': filled,
            'remaining': remaining,
            'avgPrice': avgFillPrice
        }
        logging.info(f"🔄 Order {orderId}: {status}, Filled: {filled}, Remaining: {remaining}, Price: ${avgFillPrice}")

**Contract and Order Creation Functions**

In [ ]:
def create_contract(symbol, secType='STK', exchange='SMART', currency='USD'):
    """Create a contract object for a specific security"""
    contract = Contract()
    contract.symbol = symbol.upper()
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency
    return contract

def create_order(action, quantity, order_type='MKT', limit_price=None):
    """Create an enhanced order object with more options"""
    order = Order()
    order.action = action.upper()
    order.totalQuantity = quantity
    order.orderType = order_type

    if order_type == 'LMT' and limit_price:
        order.lmtPrice = limit_price

    # Add some protective settings
    order.transmit = True
    order.eTradeOnly = False
    order.firmQuoteOnly = False

    return order

**Strategy Calculation**

In [ ]:
def enhanced_calculate_triple_sma(data, sma20_period=20, sma50_period=50, sma200_period=200,
                                 rsi_period=14, volume_threshold=1.2):
    """
    Enhanced Triple SMA strategy with RSI and volume confirmation
    """
    # Convert data to DataFrame
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close', 'volume'])

    # Convert date strings to datetime objects if needed
    if isinstance(df['date'].iloc[0], str):
        try:
            df['date'] = pd.to_datetime(df['date'], format='%Y%m%d %H:%M:%S')
        except:
            df['date'] = pd.to_datetime(df['date'])

    # Calculate SMAs
    df['sma20'] = df['close'].rolling(window=sma20_period).mean()
    df['sma50'] = df['close'].rolling(window=sma50_period).mean()
    df['sma200'] = df['close'].rolling(window=sma200_period).mean()

    # Calculate RSI for additional confirmation
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))

    # Calculate volume confirmation
    df['volume_ma'] = df['volume'].rolling(window=20).mean()
    df['volume_ratio'] = df['volume'] / df['volume_ma']

    # Calculate volatility (ATR approximation)
    df['high_low'] = df['high'] - df['low']
    df['high_close'] = abs(df['high'] - df['close'].shift())
    df['low_close'] = abs(df['low'] - df['close'].shift())
    df['atr'] = df[['high_low', 'high_close', 'low_close']].max(axis=1).rolling(window=14).mean()

    # Enhanced signal generation with multiple confirmations
    df['signal'] = 0
    df['signal_strength'] = 0
    df['stop_loss'] = 0

    for i in range(sma200_period, len(df)):
        price = df['close'].iloc[i]
        sma20 = df['sma20'].iloc[i]
        sma50 = df['sma50'].iloc[i]
        sma200 = df['sma200'].iloc[i]
        rsi = df['rsi'].iloc[i]
        vol_ratio = df['volume_ratio'].iloc[i]
        atr = df['atr'].iloc[i]

        # Strong uptrend conditions with confirmations
        if (price > sma20 > sma50 > sma200 and  # Price above all SMAs in order
            rsi > 35 and rsi < 75 and  # RSI not in extreme zones
            vol_ratio > volume_threshold and  # Volume confirmation
            (price - sma20) / sma20 < 0.05):  # Not too far from SMA20

            df['signal'].iloc[i] = 1
            df['signal_strength'].iloc[i] = calculate_signal_strength(price, sma20, sma50, sma200, rsi, vol_ratio)
            df['stop_loss'].iloc[i] = price - (2 * atr)  # ATR-based stop loss

        # Strong downtrend conditions with confirmations
        elif (price < sma20 < sma50 < sma200 and  # Price below all SMAs in order
              rsi > 25 and rsi < 65 and  # RSI not in extreme zones
              vol_ratio > volume_threshold and  # Volume confirmation
              (sma20 - price) / sma20 < 0.05):  # Not too far from SMA20

            df['signal'].iloc[i] = -1
            df['signal_strength'].iloc[i] = calculate_signal_strength(price, sma20, sma50, sma200, rsi, vol_ratio)
            df['stop_loss'].iloc[i] = price + (2 * atr)  # ATR-based stop loss

    # Calculate position changes (1 for new buy, -1 for new sell, 0 for hold)
    df['position'] = df['signal'].diff()

    return df

def calculate_signal_strength(price, sma20, sma50, sma200, rsi, vol_ratio):
    """Calculate signal strength based on multiple factors"""
    # SMA alignment strength
    sma_spread = abs(sma20 - sma200) / sma200 * 100

    # RSI strength (distance from neutral)
    rsi_strength = abs(50 - rsi) / 50 * 100

    # Volume strength
    vol_strength = min(vol_ratio * 50, 100)

    # Combine factors (weighted average)
    total_strength = (sma_spread * 0.4 + rsi_strength * 0.3 + vol_strength * 0.3)

    return min(total_strength, 100)

**Visualization**

In [ ]:
def enhanced_visualize_triple_sma_strategy(df, symbol):
    """
    Enhanced visualization with multiple subplots and indicators
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Enhanced Triple SMA Strategy Analysis for {symbol}', fontsize=16, fontweight='bold')

    # Main price chart with SMAs
    ax1.plot(df['date'], df['close'], label='Close Price', alpha=0.8, linewidth=1.5, color='black')
    ax1.plot(df['date'], df['sma20'], label='SMA 20', color='blue', linewidth=2, alpha=0.8)
    ax1.plot(df['date'], df['sma50'], label='SMA 50', color='orange', linewidth=2, alpha=0.8)
    ax1.plot(df['date'], df['sma200'], label='SMA 200', color='red', linewidth=2, alpha=0.8)

    # Plot buy/sell signals with strength indication
    strong_buys = df[(df['position'] > 0) & (df['signal_strength'] > 60)]
    weak_buys = df[(df['position'] > 0) & (df['signal_strength'] <= 60)]
    strong_sells = df[(df['position'] < 0) & (df['signal_strength'] > 60)]
    weak_sells = df[(df['position'] < 0) & (df['signal_strength'] <= 60)]

    ax1.scatter(strong_buys['date'], strong_buys['close'], marker='^',
               color='darkgreen', s=200, label='Strong Buy Signal', zorder=5, edgecolors='white')
    ax1.scatter(weak_buys['date'], weak_buys['close'], marker='^',
               color='lightgreen', s=120, label='Weak Buy Signal', zorder=5, edgecolors='white')
    ax1.scatter(strong_sells['date'], strong_sells['close'], marker='v',
               color='darkred', s=200, label='Strong Sell Signal', zorder=5, edgecolors='white')
    ax1.scatter(weak_sells['date'], weak_sells['close'], marker='v',
               color='lightcoral', s=120, label='Weak Sell Signal', zorder=5, edgecolors='white')

    ax1.set_title('Price Chart with SMA and Signals')
    ax1.set_ylabel('Price ($)')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)

    # RSI subplot
    ax2.plot(df['date'], df['rsi'], color='purple', linewidth=2)
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
    ax2.axhline(y=50, color='gray', linestyle='-', alpha=0.5, label='Neutral')
    ax2.fill_between(df['date'], 30, 70, alpha=0.1, color='gray')

    # Highlight RSI buy/sell zones
    overbought = df['rsi'] > 70
    oversold = df['rsi'] < 30
    ax2.fill_between(df['date'], df['rsi'], 70, where=overbought, alpha=0.3, color='red')
    ax2.fill_between(df['date'], df['rsi'], 30, where=oversold, alpha=0.3, color='green')

    ax2.set_title('RSI (14-period)')
    ax2.set_ylabel('RSI')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 100)

    # Volume analysis
    colors = ['red' if vol < 1 else 'green' for vol in df['volume_ratio']]
    ax3.bar(df['date'], df['volume'], alpha=0.6, color=colors)
    ax3.plot(df['date'], df['volume_ma'], color='blue', linewidth=2, label='Volume MA')
    ax3.set_title('Volume Analysis')
    ax3.set_ylabel('Volume')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # Signal strength over time
    ax4.plot(df['date'], df['signal_strength'], color='darkorange', linewidth=2)
    ax4.fill_between(df['date'], df['signal_strength'], alpha=0.3, color='orange')
    ax4.axhline(y=60, color='red', linestyle='--', alpha=0.7, label='Strong Signal Threshold')
    ax4.set_title('Signal Strength Over Time')
    ax4.set_ylabel('Signal Strength (%)')
    ax4.set_xlabel('Date')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    ax4.set_ylim(0, 100)

    plt.tight_layout()
    plt.show()

**Risk Management Functions**

In [ ]:
def calculate_position_size(account_value, risk_per_trade=0.02, stock_price=100, stop_loss_distance=5):
    """
    Calculate position size based on risk management principles
    """
    # Maximum dollar amount to risk per trade
    max_risk_amount = account_value * risk_per_trade

    # Calculate position size based on stop loss distance
    if stop_loss_distance > 0:
        position_size = max_risk_amount / stop_loss_distance
        max_shares = int(position_size)
    else:
        # Fallback: use percentage of account value
        max_shares = int((account_value * 0.1) / stock_price)

    return max_shares

def get_account_summary(app):
    """Get comprehensive account information"""
    try:
        app.reqAccountSummary(9001, "All", "$LEDGER")
        time.sleep(3)  # Wait for data
        return app.account_info
    except Exception as e:
        logging.error(f"Error getting account summary: {e}")
        return {}

**Testing Function**

In [ ]:
def enhanced_test_triple_sma_strategy(host, port, symbol='AAPL', show_visualization=True):
    """
    Enhanced strategy testing with comprehensive analysis
    """
    logging.info(f"🚀 Starting enhanced strategy test for {symbol}")

    # Initialize API connection
    app = EnhancedIBapi()

    try:
        # Connect to TWS
        app.connect(host, port, 0)
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()

        # Wait for connection
        connection_timeout = 10
        start_time = time.time()
        while not app.connection_status and time.time() - start_time < connection_timeout:
            time.sleep(0.5)

        if not app.connection_status:
            logging.error("❌ Failed to connect to TWS. Check your ngrok tunnel and TWS settings.")
            return None

        # Get account information
        account_info = get_account_summary(app)
        if account_info:
            logging.info(f"💰 Account connected successfully")

        # Create contract for the symbol
        contract = create_contract(symbol)

        # Request historical data (2 years for sufficient SMA calculation)
        app.data = []
        app.data_received = False

        logging.info(f"📊 Requesting historical data for {symbol}...")
        app.reqHistoricalData(
            reqId=1,
            contract=contract,
            endDateTime='',
            durationStr='2 Y',
            barSizeSetting='1 day',
            whatToShow='MIDPOINT',
            useRTH=1,
            formatDate=1,
            keepUpToDate=False,
            chartOptions=[]
        )

        # Wait for data to be received
        data_timeout = 20
        start_time = time.time()
        while not app.data_received and time.time() - start_time < data_timeout:
            time.sleep(0.5)

        # Disconnect
        app.disconnect()

        # Process data and apply enhanced strategy
        if len(app.data) >= 200:
            logging.info(f"✅ Received {len(app.data)} bars of historical data for {symbol}")

            # Calculate Enhanced Triple SMA and generate signals
            df = enhanced_calculate_triple_sma(app.data)

            # Visualize the strategy if requested
            if show_visualization:
                enhanced_visualize_triple_sma_strategy(df, symbol)

            # Calculate comprehensive performance metrics
            analyze_strategy_performance(df, symbol, account_info)

            return df

        else:
            logging.error(f"❌ Insufficient data received: {len(app.data)} bars (need at least 200)")
            return None

    except Exception as e:
        logging.error(f"❌ Error in strategy testing: {e}")
        return None

def analyze_strategy_performance(df, symbol, account_info):
    """Analyze and display comprehensive strategy performance"""

    signals = df[df['position'] != 0].copy()
    current = df.iloc[-1]

    print(f"\n{'='*60}")
    print(f"📈 ENHANCED STRATEGY ANALYSIS FOR {symbol}")
    print(f"{'='*60}")

    if len(signals) > 0:
        # Signal Analysis
        buy_signals = signals[signals['position'] > 0]
        sell_signals = signals[signals['position'] < 0]
        strong_signals = signals[signals['signal_strength'] > 60]

        print(f"\n📊 SIGNAL SUMMARY:")
        print(f"   Total Signals Generated: {len(signals)}")
        print(f"   Buy Signals: {len(buy_signals)}")
        print(f"   Sell Signals: {len(sell_signals)}")
        print(f"   Strong Signals (>60%): {len(strong_signals)}")

        if len(strong_signals) > 0:
            avg_strength = strong_signals['signal_strength'].mean()
            print(f"   Average Strong Signal Strength: {avg_strength:.1f}%")

        # Current Market Status
        print(f"\n📊 CURRENT MARKET STATUS:")
        print(f"   Current Price: ${current['close']:.2f}")
        print(f"   SMA 20: ${current['sma20']:.2f}")
        print(f"   SMA 50: ${current['sma50']:.2f}")
        print(f"   SMA 200: ${current['sma200']:.2f}")
        print(f"   RSI: {current['rsi']:.1f}")

        # Current Signal
        signal_text = {1: "🟢 BUY/HOLD (Strong Uptrend)",
                      -1: "🔴 SELL/SHORT (Strong Downtrend)",
                      0: "🟡 NEUTRAL (No Clear Trend)"}
        print(f"   Current Signal: {signal_text.get(current['signal'], 'Unknown')}")

        if current['signal'] != 0:
            print(f"   Signal Strength: {current['signal_strength']:.1f}%")
            print(f"   Suggested Stop Loss: ${current['stop_loss']:.2f}")

        # Risk Management Analysis
        if account_info and 'NetLiquidation' in account_info:
            account_value = float(account_info['NetLiquidation'])

            # Calculate position sizing
            if current['signal'] != 0 and current['stop_loss'] > 0:
                stop_distance = abs(current['close'] - current['stop_loss'])
                recommended_shares = calculate_position_size(
                    account_value, 0.02, current['close'], stop_distance
                )
            else:
                recommended_shares = calculate_position_size(
                    account_value, 0.02, current['close'], current['close'] * 0.05
                )

            investment_amount = recommended_shares * current['close']

            print(f"\n💰 RISK MANAGEMENT RECOMMENDATIONS:")
            print(f"   Account Value: ${account_value:,.2f}")
            print(f"   Risk per Trade (2%): ${account_value * 0.02:,.2f}")
            print(f"   Recommended Position: {recommended_shares:,} shares")
            print(f"   Investment Amount: ${investment_amount:,.2f}")
            print(f"   Portfolio Allocation: {(investment_amount/account_value)*100:.1f}%")

        # Recent Performance
        recent_signals = signals.tail(5)
        if len(recent_signals) > 0:
            print(f"\n🕒 RECENT SIGNALS (Last 5):")
            for idx, row in recent_signals.iterrows():
                signal_type = "BUY" if row['position'] > 0 else "SELL"
                date_str = row['date'].strftime('%Y-%m-%d') if hasattr(row['date'], 'strftime') else str(row['date'])
                print(f"   {date_str}: {signal_type} at ${row['close']:.2f} (Strength: {row['signal_strength']:.1f}%)")

    else:
        print(f"\n⚠️  No trading signals generated for {symbol} with current parameters.")
        print(f"   Current Price: ${current['close']:.2f}")
        print(f"   Market may be in a sideways/consolidation phase.")

    print(f"\n{'='*60}")


**Trade Execution**

In [ ]:
def enhanced_execute_trade(host, port, symbol, action, quantity, order_type='MKT', limit_price=None):
    """
    Enhanced trade execution with better error handling and confirmation
    """
    logging.info(f"🎯 Preparing to execute {action} order for {quantity} shares of {symbol}")

    app = EnhancedIBapi()

    try:
        # Connect to TWS
        app.connect(host, port, 0)
        api_thread = threading.Thread(target=app.run, daemon=True)
        api_thread.start()

        # Wait for connection
        time.sleep(3)

        if not app.connection_status:
            logging.error("❌ Failed to connect to TWS for trade execution")
            return False

        # Create contract and order
        contract = create_contract(symbol)
        order = create_order(action, quantity, order_type, limit_price)

        # Get next order ID
        if app.next_order_id:
            order_id = app.next_order_id
        else:
            logging.error("❌ No valid order ID received")
            app.disconnect()
            return False

        # Place the order
        app.placeOrder(order_id, contract, order)

        order_details = f"{action} {quantity} shares of {symbol}"
        if order_type == 'LMT':
            order_details += f" at ${limit_price}"

        logging.info(f"✅ Order placed: {order_details}")

        # Wait for order confirmation
        time.sleep(5)

        # Check order status
        if order_id in app.order_status:
            status_info = app.order_status[order_id]
            logging.info(f"📋 Order Status: {status_info['status']}")
            if status_info['filled'] > 0:
                logging.info(f"✅ Filled: {status_info['filled']} shares at ${status_info['avgPrice']:.2f}")

        app.disconnect()
        return True

    except Exception as e:
        logging.error(f"❌ Error executing trade: {e}")
        return False

**Main Interactive Interface**

In [ ]:
def run_enhanced_trading_system():
    """Main function to run the enhanced trading system"""

    print("🚀 Enhanced IBKR Trading System v2.0")
    print("=" * 50)
    print("Features:")
    print("✅ Enhanced Triple SMA with RSI and Volume confirmation")
    print("✅ Advanced risk management and position sizing")
    print("✅ Comprehensive performance analysis")
    print("✅ Multiple chart visualizations")
    print("✅ Signal strength calculation")
    print("=" * 50)

    # Get connection details
    ngrok_host = input("🌐 Enter your ngrok host (e.g., 0.tcp.ngrok.io): ").strip()

    while True:
        try:
            ngrok_port = int(input("🔌 Enter your ngrok port: ").strip())
            break
        except ValueError:
            print("❌ Please enter a valid port number")

    print(f"\n🔗 Connection: {ngrok_host}:{ngrok_port}")

    while True:
        print(f"\n{'='*50}")
        print("📋 ENHANCED TRADING SYSTEM MENU")
        print("=" * 50)
        print("1. 📊 Test Enhanced Strategy (with analysis)")
        print("2. 🎯 Execute Trade")
        print("3. 💰 Quick Account Check")
        print("4. 📈 Multi-Symbol Analysis")
        print("5. ❌ Exit")
        print("=" * 50)

        choice = input("Enter your choice (1-5): ").strip()

        if choice == "1":
            # Enhanced strategy testing
            symbol = input("📈 Enter symbol to analyze (default: AAPL): ").strip().upper() or "AAPL"
            show_viz = input("🎨 Show visualizations? (y/n, default: y): ").strip().lower()
            show_visualization = show_viz != 'n'

            print(f"\n🔍 Running enhanced analysis for {symbol}...")
            df = enhanced_test_triple_sma_strategy(ngrok_host, ngrok_port, symbol, show_visualization)

            if df is not None:
                latest_signal = df['signal'].iloc[-1]
                latest_strength = df['signal_strength'].iloc[-1]

                print(f"\n🎯 RECOMMENDATION:")
                if latest_signal == 1:
                    print(f"   💚 BUY/HOLD {symbol} (Strength: {latest_strength:.1f}%)")
                elif latest_signal == -1:
                    print(f"   ❤️ SELL/SHORT {symbol} (Strength: {latest_strength:.1f}%)")
                else:
                    print(f"   💛 NEUTRAL for {symbol} (Wait for clearer signal)")

        elif choice == "2":
            # Enhanced trade execution
            symbol = input("📈 Enter symbol to trade (default: AAPL): ").strip().upper() or "AAPL"

            print("📋 Available actions:")
            print("   - BUY: Open long position")
            print("   - SELL: Close long position")

            action = input("🎯 Enter action (BUY/SELL): ").strip().upper()

            if action not in ["BUY", "SELL"]:
                print("❌ Invalid action. Must be BUY or SELL.")
                continue

            try:
                quantity = int(input("📊 Enter quantity: ").strip())
            except ValueError:
                print("❌ Please enter a valid quantity.")
                continue

            order_type = input("📋 Order type (MKT/LMT, default: MKT): ").strip().upper() or "MKT"
            limit_price = None

            if order_type == "LMT":
                try:
                    limit_price = float(input("💰 Enter limit price: ").strip())
                except ValueError:
                    print("❌ Please enter a valid limit price.")
                    continue

            # Confirmation
            order_summary = f"{action} {quantity} shares of {symbol}"
            if order_type == "LMT":
                order_summary += f" at ${limit_price}"

            confirm = input(f"✅ Confirm order: {order_summary}? (y/n): ").strip().lower()

            if confirm == 'y':
                print(f"\n🎯 Executing {order_summary}...")
                success = enhanced_execute_trade(ngrok_host, ngrok_port, symbol, action, quantity, order_type, limit_price)
                if success:
                    print(f"✅ Order for {symbol} successfully submitted!")
                else:
                    print(f"❌ Failed to submit order for {symbol}")
            else:
                print("❌ Order cancelled.")

        elif choice == "3":
            # Quick account check
            print("\n💰 Getting account information...")
            app = EnhancedIBapi()

            try:
                app.connect(ngrok_host, ngrok_port, 0)
                api_thread = threading.Thread(target=app.run, daemon=True)
                api_thread.start()
                time.sleep(3)

                if app.connection_status:
                    account_info = get_account_summary(app)
                    app.reqPositions()  # Get current positions
                    time.sleep(2)
                    app.disconnect()

                    if account_info:
                        print(f"\n📊 ACCOUNT SUMMARY:")
                        print(f"   Net Liquidation: ${float(account_info.get('NetLiquidation', 0)):,.2f}")
                        print(f"   Available Funds: ${float(account_info.get('AvailableFunds', 0)):,.2f}")
                        print(f"   Buying Power: ${float(account_info.get('BuyingPower', 0)):,.2f}")

                        if app.positions:
                            print(f"\n📈 CURRENT POSITIONS:")
                            for symbol, pos_info in app.positions.items():
                                print(f"   {symbol}: {pos_info['position']} shares @ ${pos_info['avgCost']:.2f}")
                        else:
                            print(f"\n📈 No current positions")
                    else:
                        print("❌ Could not retrieve account information")
                else:
                    print("❌ Failed to connect for account check")

            except Exception as e:
                print(f"❌ Error checking account: {e}")

        elif choice == "4":
            # Multi-symbol analysis
            symbols_input = input("📈 Enter symbols separated by commas (e.g., AAPL,MSFT,GOOGL): ").strip().upper()
            symbols = [s.strip() for s in symbols_input.split(',') if s.strip()]

            if not symbols:
                print("❌ No valid symbols entered")
                continue

            print(f"\n🔍 Analyzing {len(symbols)} symbols...")
            results = {}

            for symbol in symbols:
                print(f"\n📊 Analyzing {symbol}...")
                df = enhanced_test_triple_sma_strategy(ngrok_host, ngrok_port, symbol, show_visualization=False)

                if df is not None:
                    current = df.iloc[-1]
                    results[symbol] = {
                        'signal': current['signal'],
                        'strength': current['signal_strength'],
                        'price': current['close'],
                        'rsi': current['rsi']
                    }
                else:
                    results[symbol] = None

            # Display results summary
            print(f"\n{'='*60}")
            print(f"📋 MULTI-SYMBOL ANALYSIS SUMMARY")
            print(f"{'='*60}")
            print(f"{'Symbol':<8} {'Signal':<12} {'Strength':<10} {'Price':<10} {'RSI':<6}")
            print(f"{'-'*60}")

            for symbol, data in results.items():
                if data:
                    signal_text = {1: "BUY", -1: "SELL", 0: "NEUTRAL"}[data['signal']]
                    print(f"{symbol:<8} {signal_text:<12} {data['strength']:<10.1f} ${data['price']:<9.2f} {data['rsi']:<6.1f}")
                else:
                    print(f"{symbol:<8} {'ERROR':<12} {'-':<10} {'-':<10} {'-':<6}")

            # Show best opportunities
            valid_results = {k: v for k, v in results.items() if v is not None}
            if valid_results:
                # Best buy opportunities
                buy_opportunities = {k: v for k, v in valid_results.items() if v['signal'] == 1}
                if buy_opportunities:
                    best_buy = max(buy_opportunities.items(), key=lambda x: x[1]['strength'])
                    print(f"\n🟢 BEST BUY OPPORTUNITY: {best_buy[0]} (Strength: {best_buy[1]['strength']:.1f}%)")

                # Best sell opportunities
                sell_opportunities = {k: v for k, v in valid_results.items() if v['signal'] == -1}
                if sell_opportunities:
                    best_sell = max(sell_opportunities.items(), key=lambda x: x[1]['strength'])
                    print(f"🔴 BEST SELL OPPORTUNITY: {best_sell[0]} (Strength: {best_sell[1]['strength']:.1f}%)")

        elif choice == "5":
            print("👋 Exiting Enhanced Trading System. Happy trading!")
            break

        else:
            print("❌ Invalid choice. Please enter 1-5.")

**Additional Utility Functions**

In [ ]:
def backup_get_stock_data(symbol):
    """Backup function to get stock data using yfinance if IBKR fails"""
    try:
        import yfinance as yf
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period="2y")

        if len(hist) > 0:
            # Convert to IBKR format
            data = []
            for idx, row in hist.iterrows():
                data.append([
                    idx.strftime('%Y%m%d %H:%M:%S'),
                    row['Open'], row['High'], row['Low'], row['Close'], row['Volume']
                ])
            return data
        return None
    except:
        return None

def save_analysis_results(df, symbol, filename=None):
    """Save analysis results to CSV file"""
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{symbol}_analysis_{timestamp}.csv"

    try:
        # Select key columns for export
        export_df = df[['date', 'close', 'sma20', 'sma50', 'sma200', 'rsi',
                       'signal', 'signal_strength', 'position']].copy()
        export_df.to_csv(filename, index=False)
        print(f"💾 Analysis results saved to {filename}")
        return filename
    except Exception as e:
        print(f"❌ Error saving results: {e}")
        return None

def get_market_hours_info():
    """Display market hours information"""
    from datetime import datetime
    import pytz

    # US market hours (Eastern Time)
    est = pytz.timezone('US/Eastern')
    now_est = datetime.now(est)

    print(f"\n🕒 MARKET HOURS INFO:")
    print(f"   Current EST Time: {now_est.strftime('%Y-%m-%d %H:%M:%S %Z')}")
    print(f"   Regular Hours: 9:30 AM - 4:00 PM EST")
    print(f"   Pre-Market: 4:00 AM - 9:30 AM EST")
    print(f"   After-Hours: 4:00 PM - 8:00 PM EST")

    # Check if market is currently open
    market_open = now_est.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now_est.replace(hour=16, minute=0, second=0, microsecond=0)

    if market_open <= now_est <= market_close and now_est.weekday() < 5:
        print(f"   Status: 🟢 MARKET IS OPEN")
    else:
        print(f"   Status: 🔴 MARKET IS CLOSED")

**System Initialization and Help**

In [ ]:
def display_system_help():
    """Display comprehensive help information"""
    print(f"\n{'='*70}")
    print(f"📚 ENHANCED IBKR TRADING SYSTEM - HELP GUIDE")
    print(f"{'='*70}")

    print(f"\n🔧 SETUP REQUIREMENTS:")
    print(f"   1. IBKR TWS or Gateway running on your local machine")
    print(f"   2. API connections enabled in TWS/Gateway")
    print(f"   3. ngrok tunnel exposing TWS port (usually 7497 for paper trading)")
    print(f"   4. Paper trading account recommended for testing")

    print(f"\n📊 STRATEGY EXPLANATION:")
    print(f"   • Triple SMA: Uses 20, 50, and 200-day moving averages")
    print(f"   • RSI Filter: Relative Strength Index for momentum confirmation")
    print(f"   • Volume Analysis: Confirms signals with volume spikes")
    print(f"   • Signal Strength: Calculates confidence level (0-100%)")

    print(f"\n🎯 SIGNAL INTERPRETATION:")
    print(f"   • BUY: Price > SMA20 > SMA50 > SMA200 + confirmations")
    print(f"   • SELL: Price < SMA20 < SMA50 < SMA200 + confirmations")
    print(f"   • NEUTRAL: Mixed signals or insufficient confirmation")
    print(f"   • Strong signals: >60% strength rating")

    print(f"\n💰 RISK MANAGEMENT:")
    print(f"   • Position sizing: 2% account risk per trade (adjustable)")
    print(f"   • Stop losses: Based on ATR (Average True Range)")
    print(f"   • Portfolio allocation: Recommended limits provided")

    print(f"\n🚨 IMPORTANT DISCLAIMERS:")
    print(f"   • This is for educational purposes only")
    print(f"   • Past performance doesn't guarantee future results")
    print(f"   • Always test strategies in paper trading first")
    print(f"   • Consider consulting a financial advisor")

    print(f"\n🔗 NGROK SETUP TIPS:")
    print(f"   1. Install ngrok: https://ngrok.com/")
    print(f"   2. Run: ngrok tcp 7497 (or your TWS port)")
    print(f"   3. Use the forwarding address in this system")
    print(f"   4. Keep ngrok running during trading sessions")

    print(f"{'='*70}")

def system_check():
    """Perform system check to ensure all components are working"""
    print(f"\n🔍 PERFORMING SYSTEM CHECK...")
    print(f"{'='*50}")

    checks = {
        "pandas": False,
        "numpy": False,
        "matplotlib": False,
        "ibapi": False,
        "threading": False,
        "datetime": False
    }

    # Check imports
    try:
        import pandas as pd
        checks["pandas"] = True
        print("✅ pandas: OK")
    except ImportError:
        print("❌ pandas: MISSING")

    try:
        import numpy as np
        checks["numpy"] = True
        print("✅ numpy: OK")
    except ImportError:
        print("❌ numpy: MISSING")

    try:
        import matplotlib.pyplot as plt
        checks["matplotlib"] = True
        print("✅ matplotlib: OK")
    except ImportError:
        print("❌ matplotlib: MISSING")

    try:
        from ibapi.client import EClient
        checks["ibapi"] = True
        print("✅ IBKR API: OK")
    except ImportError:
        print("❌ IBKR API: MISSING")

    try:
        import threading
        checks["threading"] = True
        print("✅ threading: OK")
    except ImportError:
        print("❌ threading: MISSING")

    try:
        import datetime
        checks["datetime"] = True
        print("✅ datetime: OK")
    except ImportError:
        print("❌ datetime: MISSING")

    print(f"{'='*50}")

    if all(checks.values()):
        print("🎉 ALL SYSTEMS GO! Ready for trading.")
        return True
    else:
        missing = [k for k, v in checks.items() if not v]
        print(f"⚠️  Missing components: {', '.join(missing)}")
        print("Please install missing packages before proceeding.")
        return False


**Main Execution Cell**

In [ ]:
print("🚀 ENHANCED IBKR TRADING SYSTEM v2.0 LOADED!")
print("=" * 50)

# Perform system check
if system_check():
    print(f"\n🎯 Ready to start trading!")
    print(f"\nAvailable functions:")
    print(f"   • run_enhanced_trading_system() - Start main interface")
    print(f"   • display_system_help() - Show detailed help")
    print(f"   • get_market_hours_info() - Check market status")

    # Ask if user wants to start immediately
    start_now = input(f"\n🚀 Start the enhanced trading system now? (y/n): ").strip().lower()
    if start_now == 'y':
        run_enhanced_trading_system()
    else:
        print(f"\n💡 Call run_enhanced_trading_system() when ready to start!")
        print(f"💡 Call display_system_help() for detailed information!")
else:
    print(f"\n❌ System check failed. Please fix missing components first.")
    print(f"💡 Try running the package installation cells again.")

# Display quick start guide
print(f"\n{'='*60}")
print(f"📋 QUICK START GUIDE:")
print(f"1. Ensure TWS/Gateway is running with API enabled")
print(f"2. Set up ngrok tunnel: ngrok tcp 7497")
print(f"3. Run: run_enhanced_trading_system()")
print(f"4. Enter your ngrok connection details")
print(f"5. Choose option 1 to test strategy")
print(f"6. Analyze results before live trading")
print(f"{'='*60}")

# Show current market status
get_market_hours_info()